### 교차검증
- 08_와인데이터

#### 데이터 불러오기, 전처리

In [2]:
import pandas as pd

file = './data/08_wine.csv'
wine = pd.read_csv(file)
wine.head(5)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [6]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

print(data.shape)
print(target.shape)

(6497, 3)
(6497,)


In [9]:
# 훈련데이터 : 검증데이터 : 테스트데이터로 쪼개기
# 보통 6 : 2 : 2

from sklearn.model_selection import train_test_split

# 1. 훈련데이터와 테스트데이터 8:2로 나누기
train_input, test_input, train_target, test_target = train_test_split(data,
                                                                      target,
                                                                      test_size=0.2,
                                                                      random_state=42)

# 2. 훈련데이터를 이용해서 검증데이터 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input,
                                                                train_target,
                                                                test_size=0.2,
                                                                random_state=42)


#### 결정트리로 훈련 및 평가

In [17]:
# sub 및 val 데이터 이용

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [16]:
print(dt.score(test_input, test_target))

0.8592307692307692


#### 교차검증
- 훈련데이터와 검증데이터를 구분할 필요없이 내부적으로 구분해서 사용
- 내부적으로 구분에 사용하는 데이터는 훈련(train)데이터
- 데이터 준비는 기존처럼 훈련과 테스트데이터만 준비
- 검증데이터는 별도로 준비 안해도 됨
- 
- 주요 키워드
- 3-폴드(fold) 교차검증
    - 훈련데이터를 세부분으로 나눠서 데이터를 쪼개고 수행
    - 구간마다 훈련데이터를 검증데이터로 바꿔가면서 수행
    - k-폴드 교차검증 또는 k-겹 교차검증이라고 함
- 보통 5-폴드 교차검증 또는 10-폴드 교차검증을 사용
    - 훈련데이터로 80% 이상을 모두 모델 훈련에 사용 가능

In [18]:
# 사용하는 데이터 확인
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [33]:
# 교차검증 모듈 : cross_validate
from sklearn.model_selection import cross_validate

# dt : 결정트리 훈련모델(다른 모델을 사용한 경우 해당 모델)
# 두번째값 : 훈련데이터
# 세번째값 : 검증데이터
scores = cross_validate(dt, train_input, train_target)
print(scores)

# 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('test_score 평균 =', scores['test_score'].mean())

{'fit_time': array([0.01198602, 0.01004553, 0.01012897, 0.00962973, 0.00905013]), 'score_time': array([0.00100017, 0.00192237, 0.00186372, 0.00100708, 0.0009234 ]), 'test_score': array([0.87596154, 0.85096154, 0.87487969, 0.85466795, 0.83541867])}
test_score 평균 = 0.8583778781372622


In [34]:
# 훈련데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# StratifiedKFold

from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음

spliter = StratifiedKFold(n_splits=10,     # fold 갯수
                          shuffle=True,    # 섞기
                          random_state=42) #

scores = cross_validate(dt, train_input, train_target, cv = spliter)

print(scores)

# 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('test_score 평균 =', scores['test_score'].mean())

{'fit_time': array([0.0137248 , 0.01395011, 0.01224422, 0.01199698, 0.01196504,
       0.01264071, 0.01196837, 0.01197314, 0.01123476, 0.01196575]), 'score_time': array([0.00103092, 0.00099874, 0.00072145, 0.001966  , 0.00099778,
       0.00032473, 0.00219297, 0.00198841, 0.00119686, 0.0009973 ]), 'test_score': array([0.83653846, 0.88269231, 0.84038462, 0.85576923, 0.85961538,
       0.86730769, 0.86346154, 0.85741811, 0.8477842 , 0.87668593])}
test_score 평균 = 0.8587657477397362


#### 하이퍼파라미터 튜닝(AutoML)
- 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
- 이런 개념을 AutoML이라고 함
- 사용 패키지 : sklearn.model_selection
- 사용 클래스 : GridSearchCV
- 그리드 서치라고도 함

#### 결정트리의 max_depth(트리의 깊이)
- 그리드서치(GridSearchCV) 객체(모델) 사용

In [38]:
from sklearn.model_selection import GridSearchCV

# 찾을 파라미터는 딕셔너리로 
# 실제 파라미터 변수이름 그대로, 찾을 범위 지정
params = {'max_depth':range(5,20,1)}

# 객체(모델) 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), # 훈련모델
                  params,                                  # 찾을 하이퍼파라미터 값들(딕셔너리로 정의)
                  n_jobs=-1)                               # CPU 코어 갯수(-1은 모든 코어 사용, 병렬 처리)

# 훈련시키기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [39]:
# 그리드서치의 결과
# 하이퍼파라미터의 찾은 값 알려줌
# 중요 : 훈련모델이 생성되어 반환됨
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [41]:
# 그리드서치를 통해 
# 평가해보기, 훈련데이터 전체 사용
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9003271117952665


In [42]:
print(gs.best_params_)

{'max_depth': 8}


In [44]:
print(gs.cv_results_.keys())

# dict_keys(['mean_fit_time',   
#            'std_fit_time',
#            'mean_score_time',
#            'std_score_time',
#            'param_max_depth',
#            'params',
#            'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 
#            'mean_test_score',
#            'std_test_score',
#            'rank_test_score'])

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [45]:
# params 값 확인하기
print(gs.cv_results_['params'])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]


In [46]:
# 1폴드 값 조회하기
print(gs.cv_results_['split0_test_score'])

[0.84711538 0.84807692 0.85769231 0.85288462 0.85769231 0.84519231
 0.85865385 0.86730769 0.86538462 0.86826923 0.86730769 0.86442308
 0.86346154 0.86634615 0.87211538]


In [47]:
# 훈련모델 최종 검증하기
dt.score(test_input, test_target)

0.8584615384615385

In [68]:
import pandas as pd
df = pd.DataFrame([gs.cv_results_['params'],gs.cv_results_['split0_test_score'],gs.cv_results_['split1_test_score'],gs.cv_results_['split2_test_score'],gs.cv_results_['split3_test_score'],gs.cv_results_['split4_test_score']])
df = df.transpose()
df.columns = ['max_depth','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score']
df

,max_depth,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score
0,{'max_depth': 5},0.847115,0.863462,0.880654,0.836381,0.861405
1,{'max_depth': 6},0.848077,0.854808,0.873917,0.843118,0.85948
2,{'max_depth': 7},0.857692,0.851923,0.875842,0.843118,0.847931
3,{'max_depth': 8},0.852885,0.854808,0.87873,0.846968,0.865255
4,{'max_depth': 9},0.857692,0.859615,0.87103,0.842156,0.854668
5,{'max_depth': 10},0.845192,0.847115,0.876805,0.846968,0.852743
6,{'max_depth': 11},0.858654,0.850962,0.880654,0.852743,0.848893
7,{'max_depth': 12},0.867308,0.848077,0.882579,0.848893,0.848893
8,{'max_depth': 13},0.865385,0.853846,0.873917,0.853705,0.839269
9,{'max_depth': 14},0.868269,0.851923,0.870067,0.858518,0.841193


#### 하이퍼파라미터 여러개 사용

#### 그리드서치(Grid Search)

In [74]:
import numpy as np

params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001), # 결정트리 분리시 불순도 감소 최소량
          'max_depth': range(5,20,1),                               # 결정트리 깊이
          'min_samples_split': range(2,100,10)}                      # 결정트리 노드를 나누기 위한 최소 생활 갯수

# 그리드서치 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                                         params,
                                         n_jobs= -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [75]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [77]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.892053107562055
0.8615384615384616


In [82]:
# 10-폴드 지정하기
spliter = StratifiedKFold(n_splits=7,     # fold 갯수
                          shuffle=True,    # 섞기
                          random_state=42) #

params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001), # 결정트리 분리시 불순도 감소 최소량
          'max_depth': range(5,20,1),                               # 결정트리 깊이
          'min_samples_split': range(2,100,10)}                      # 결정트리 노드를 나누기 위한 최소 생활 갯수

# 그리드서치 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                                         params,
                                         n_jobs= -1)
gs.fit(train_input, train_target)

scores = cross_validate(dt, train_input, train_target, cv = spliter)

print(scores)

# 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('test_score 평균 =', scores['test_score'].mean())

{'fit_time': array([0.00697994, 0.00782394, 0.006953  , 0.00716019, 0.00693655,
       0.00698233, 0.0069809 ]), 'score_time': array([0.00099874, 0.00115013, 0.0009985 , 0.00081682, 0.        ,
       0.        , 0.        ]), 'test_score': array([0.85868102, 0.86137281, 0.85868102, 0.86522911, 0.86522911,
       0.85849057, 0.86118598])}
test_score 평균 = 0.8612670899386444


In [83]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.892053107562055
0.8615384615384616


#### 랜덤서치(Random Search)
- 랜덤서치 사용시 아래 조건 만족해야함
- 하이퍼파라미터 값이 수치인 것들만 사용
- 범위나 간격을 미리 정하기 어려울 경우
- 너무 많은 매개변수가 있어서 그리드서치 수행시간이 오래 걸릴 경우
- 매개변수를 샘플링할때 확률분포객체를 전달
- 
- 사용 모듈 : uniform(실수값), randint(정수값)

In [89]:
from scipy.stats import uniform, randint

# 정수값 범위설정
rgen = randint(0, 10)
rgen

# 범위 내에서 임의값 출력하기(확률적으로 고르게 뽑아냄)
print(rgen.rvs(100))

[2 6 1 1 3 9 9 3 1 4 3 3 4 6 4 5 1 7 7 3 6 7 9 3 2 9 3 6 9 9 0 6 3 9 4 8 9
 1 8 0 0 7 6 3 9 3 1 3 3 6 9 0 2 6 4 4 1 5 4 4 7 1 3 9 5 5 5 8 6 7 0 9 0 0
 4 1 7 5 3 5 0 5 7 8 1 3 1 4 6 7 2 8 4 3 3 3 2 8 1 6]


In [91]:
# 실수값 범위설정
ugen = uniform(0, 1)
ugen

# 임의값 50개 조회
print(ugen.rvs(50))

[0.32362164 0.16633632 0.87343    0.24400679 0.40396064 0.71010056
 0.81770436 0.15408316 0.01461404 0.64623245 0.5327572  0.25891185
 0.47662469 0.47288101 0.31683885 0.88065496 0.97733444 0.6273423
 0.03941794 0.90585041 0.25815008 0.2226157  0.13131882 0.49467333
 0.40252741 0.95286231 0.86818142 0.419454   0.22436782 0.01314247
 0.43572707 0.47504738 0.84106749 0.60494657 0.86228035 0.98291188
 0.10601337 0.48803707 0.48669043 0.3642817  0.80634997 0.64594288
 0.63760487 0.71752059 0.29675597 0.58373807 0.41921164 0.82243388
 0.26129974 0.67165279]


In [100]:
# 하이퍼파라미터 찾을 매개변수 정의하기
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25)}

# 랜덤서치 클래스(모델) : RandomizedSearchCV()
# 사용방법은 그리드서치와 동일
# 결정트리 모델사용, 훈련반복 횟수 100회

from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                                         params,
                                         n_iter=100,
                                         n_jobs= -1,
                                         random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000217C9EE4820>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000217C9F1F790>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000217C9C785B0>},
                   random_state=42)

In [101]:
print(rs.best_params_)

{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628, 'min_samples_split': 16}


In [106]:
print(rs.cv_results_.keys())
print(rs.cv_results_['mean_test_score'].max())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])
0.8695409787517583


In [102]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.8874350586877044
0.8615384615384616
